In [1]:
import great_expectations as ge 

In [2]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark=SparkSession.builder.appName('ge').getOrCreate()

In [6]:
spark.sparkContext.setLogLevel("WARN")

In [7]:
spark

In [8]:
from pyspark.sql.functions import *

In [9]:
df=spark.read.option('header','true').csv('dataset.csv')

In [10]:
df.show()

+-----------------+----------------+---+------+-------------------+
|               ID|            name|age|gender|              email|
+-----------------+----------------+---+------+-------------------+
|                1|     Peter Poole| 30|  Male|      po@zalowpo.az|
|                2|   Herbert Payne| 46|Female|      zos@toilet.bn|
|                3|      Callie Roy| 57|  Male|kapkapor@reckeho.ye|
|                4|  Isabella Wolfe| 61|Female| atuigome@asedas.io|
|                5|     Sara Knight| 53|Female| dudnukob@ucboki.tg|
|                6| Raymond Herrera| 23|  Male|       tolpij@ni.lu|
|                7|  Jeffery Brooks| 65|  Male|   anirus@ajjuhu.mw|
|                8|  Lillie Barnett| 55|  Male|      besi@vodje.hr|
|                9|    Emma Alvarez| 21|  Male|          ze@ebo.jm|
|               10|  Estella Rhodes| 27|  Male|         ez@jewe.si|
|               11|     Eugene West| 18|Female| hohobum@janucke.vn|
|               12|     Ollie Blake| 57|  Male| 

In [12]:

import datetime

from ruamel import yaml
from great_expectations.core.batch import RuntimeBatchRequest
from great_expectations.data_context import BaseDataContext
from great_expectations.data_context.types.base import (DataContextConfig,FilesystemStoreBackendDefaults,
)
root_directory = r"C:\Users\Namita\OneDrive\Desktop\ge"



In [13]:

from great_expectations import *
#create Data Context
data_context_config = DataContextConfig(
    store_backend_defaults=FilesystemStoreBackendDefaults(
        root_directory=root_directory
    ),
)



In [14]:
data_context_config = DataContextConfig(
    validation_operators={
        "action_list_operator": {
            "class_name": "ActionListValidationOperator",
            "action_list": [
                {
                    "name": "store_validation_result",
                    "action": {"class_name": "StoreValidationResultAction"},
                },
                {
                    "name": "store_evaluation_params",
                    "action": {"class_name": "StoreEvaluationParametersAction"},
                },
                {
                    "name": "update_data_docs",
                    "action": {"class_name": "UpdateDataDocsAction"},
                },
            ],
        }
    },
    store_backend_defaults=FilesystemStoreBackendDefaults(
        root_directory=root_directory
    ),
)
context = BaseDataContext(project_config=data_context_config)

In [15]:
my_spark_datasource_config = {
    "name": "insert_your_datasource_name_here",
    "class_name": "Datasource",
    "execution_engine": {"class_name": "SparkDFExecutionEngine"},
    "data_connectors": {
        "insert_your_data_connector_name_here": {
            "module_name": "great_expectations.datasource.data_connector",
            "class_name": "RuntimeDataConnector",
            "batch_identifiers": [
                "some_key_maybe_pipeline_stage",
                "some_other_key_maybe_run_id",
            ],
        }
    },
}
 
context.test_yaml_config(yaml.dump(my_spark_datasource_config))
 
context.add_datasource(**my_spark_datasource_config)
 
batch_request = RuntimeBatchRequest(
    datasource_name="insert_your_datasource_name_here",
    data_connector_name="insert_your_data_connector_name_here",
    data_asset_name="<YOUR_MEANGINGFUL_NAME>",  # This can be anything that identifies this data_asset for you
    batch_identifiers={
        "some_key_maybe_pipeline_stage": "prod",
        "some_other_key_maybe_run_id": f"my_run_name_{datetime.date.today().strftime('%Y%m%d')}",
    },
    runtime_parameters={"batch_data": df},  # Your dataframe goes here
)
 
expectation_suite_name = "insert_your_expectation_suite_name_here"
context.create_expectation_suite(
    expectation_suite_name=expectation_suite_name, overwrite_existing=True
)
# validator = context.get_validator(
#     batch_request=batch_request,
#     expectation_suite_name=expectation_suite_name,
# )

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: SparkDFExecutionEngine
Data Connectors:
	insert_your_data_connector_name_here:RuntimeDataConnector

	Available data_asset_names (0 of 0):
		Note : RuntimeDataConnector will not have data_asset_names until they are passed in through RuntimeBatchRequest

	Unmatched data_references (0 of 0): []



{
  "ge_cloud_id": null,
  "expectation_suite_name": "insert_your_expectation_suite_name_here",
  "expectations": [],
  "meta": {
    "great_expectations_version": "0.15.36"
  },
  "data_asset_type": null
}

In [16]:
suit_lst = []
col_expectations = {"expect_column_values_to_not_be_null":{"column":"age"},"expect_compound_columns_to_be_unique":{"column":"id"}}

from great_expectations.core import ExpectationSuite, ExpectationConfiguration
for k,v in col_expectations.items():
    expectation_configuration = ExpectationConfiguration(
    expectation_type =k, kwargs = v
    )
    suit_lst.append(expectation_configuration)

suite = ExpectationSuite("custom_expec_test",expectations=suit_lst)

validator_list = []
validator = context.get_validator(batch_request= batch_request, expectation_suite= suite)

validator.save_expectation_suite(discard_failed_expectations=False)

validator_list.append(validator)
results = context.run_validation_operator("action_list_operator",assets_to_validate=validator_list)

print(results)

Calculating Metrics:   0%|          | 0/12 [00:00<?, ?it/s]

An unexpected Exception occurred during data docs rendering.  Because of this error, certain parts of data docs will not be rendered properly and/or may not appear altogether.  Please use the trace, included in this message, to diagnose and repair the underlying issue.  Detailed information follows:
        TypeError: "object of type 'NoneType' has no len()".  Traceback: "Traceback (most recent call last):
  File "C:\Users\Namita\anaconda3\lib\site-packages\great_expectations\render\renderer\content_block\content_block.py", line 107, in _render_list
    result = content_block_fn(
  File "C:\Users\Namita\anaconda3\lib\site-packages\great_expectations\render\renderer\renderer.py", line 14, in inner_func
    return renderer_fn(*args, **kwargs)
  File "C:\Users\Namita\anaconda3\lib\site-packages\great_expectations\expectations\expectation.py", line 118, in inner_func
    ] = render_func(*args, **kwargs)
  File "C:\Users\Namita\anaconda3\lib\site-packages\great_expectations\expectations\cor

{
  "run_results": {
    "ValidationResultIdentifier::custom_expec_test/20221213T184420.275666Z/20221213T184420.275666Z/68a74190e9f8d93771db1ff953848ddd": {
      "validation_result": {
        "results": [
          {
            "result": {},
            "success": false,
            "expectation_config": {
              "kwargs": {
                "column": "age",
                "batch_id": "68a74190e9f8d93771db1ff953848ddd"
              },
              "expectation_type": "expect_column_values_to_not_be_null",
              "meta": {}
            },
            "meta": {},
            "exception_info": {
              "exception_traceback": "Traceback (most recent call last):\n  File \"C:\\Users\\Namita\\anaconda3\\lib\\site-packages\\great_expectations\\execution_engine\\execution_engine.py\", line 421, in resolve_metrics\n    ] = self.resolve_metric_bundle(metric_fn_bundle=metric_fn_bundle)\n  File \"C:\\Users\\Namita\\anaconda3\\lib\\site-packages\\great_expectations\\executi